# Testing node2vec on Directed Graphs

In [ ]:
# default_exp 
from nbdev.showdoc import *
import numpy as np

%load_ext autoreload
%autoreload 2

In [ ]:
from node2vec import node2vec

We'll try 

In [ ]:
# export 
